In [1]:
!pip install dash==1.19.0  

!pip install jupyter_dash 

!pip install --upgrade plotly

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84011 sha256=8300678cc21575f4de2492ec3736589a79ab2f99d4c4e993922e37db1e12afe5
  Stored in directory: c:\users\25127\appdata\local\pip\cache\wheels\e7\27\2f\a9f9bf858843b5365bcb311a80f0b2379737caf4ec50b2ae3a
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527011 sha256=e38d4a391597cc91cb816bf281bd368967b3cf1d8c715cc865e45f8c2e47a9f6
  Stored in directory: c:\users\2


  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-py3-none-any.whl size=1014872 sha256=c513d03bcd28fd2980953354bbc5b68835b99b3a05434e6389ee2f40ecfaa0d2
  Stored in directory: c:\users\25127\appdata\local\pip\cache\wheels\08\94\09\e11ba8ae5d97e1c8ff5be0ac1f7cf42642eec6ea94ed1e4b44
  Created wheel for dash-table: filename=dash_table-4.11.2-py3-none-any.whl size=1839869 sha256=c0eab0da45b8ddd2a1dc2ed025e959ae32a7c1db970d6cc24ed92003d49f8521
  Stored in directory: c:\users\25127\appdata\local\pip\cache\wheels\cc\1d\a0\720dcf854e410eddbe44f8434f4d3a6332fdd085178eeac4f9
Successfully built dash dash-core-components dash-html-components dash-renderer dash-table
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=87287668626177d97c83c6ffb381e5d922cfedff6d860d523afaa7e49ed84c44
  Stored in directory: c:\users\25127\appdata\local\pip\cache\wheel

  Attempting uninstall: plotly
    Found existing installation: plotly 5.1.0
    Uninstalling plotly-5.1.0:
      Successfully uninstalled plotly-5.1.0


In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

In [2]:
import plotly.express as px
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'All Sites'})

for site in uniquelaunchsites:
    lsites.append({'label': site, 'value': site})  
    
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                
                                

                    
                                dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', 
                                             searchable = True , value = 'All Sites'),
                                html.Br(),

                       
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                             
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,  
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',  
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'                                         
                                    },

                                    value=[min_payload,max_payload]
                                ),
                            
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                              
                                ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]               
)
def update_graph(site_dropdown):
    if (site_dropdown == 'All Sites'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=.3,title = 'Total Success Launches By all sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),
      Input(component_id="payload_slider", component_property="value")]               
)

def update_scattergraph(site_dropdown,payload_slider):
    if (site_dropdown == 'All Sites'):
        low, high = payload_slider
        df  = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig
        
# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
                   dev_tools_hot_reload =True, threaded=True)